In [1]:
import pandas as pd
from pathlib import Path

repo = Path('..').resolve()
omxs30_path = '../omxs30_members.csv'
master_path = '../instrument_master.csv'
coverage_path = '../coverage_table_updated.csv'

omxs30 = pd.read_csv(omxs30_path, sep='\t')
master = pd.read_csv(master_path, sep='\t')
coverage = pd.read_csv(coverage_path, sep='\t')
omxs30.head(), master.head(), coverage.head() if not coverage.empty else 'no coverage yet'

(       Fullname  CCY      Last   +/-       %       Bid       Ask     Volume  \
 0       ABB Ltd  SEK    683.00   4.4  +0.64%    681.40    681.80    453,736   
 1     Addtech B  SEK    325.60   2.0  +0.61%    325.40    325.80    350,644   
 2    Alfa Laval  SEK    460.60   3.9  +0.85%    458.40    458.60    834,570   
 3  ASSA ABLOY B  SEK    355.80   0.8  +0.22%    356.20    356.40  1,924,559   
 4   AstraZeneca  SEK  1,701.00  16.5  +0.97%  1,700.00  1,700.50    283,942   
 
       Turnover Updated (CET)  
 0  308,982,606      18:00:00  
 1  114,134,399      18:00:00  
 2  384,199,339      18:00:00  
 3  685,032,972      18:00:00  
 4  480,673,375      18:00:00  ,
     CompanyName  ISIN CID  date refreshed  \
 0       ABB Ltd   NaN  S1             NaN   
 1     Addtech B   NaN  S2             NaN   
 2    Alfa Laval   NaN  S3             NaN   
 3  ASSA ABLOY B   NaN  S4             NaN   
 4   AstraZeneca   NaN  S5             NaN   
 
                                           Nasd

In [2]:
# Verify all 30 companies exist in instrument_master
missing_in_master = set(omxs30['Fullname']) - set(master['CompanyName'])
missing_in_master

set()

In [3]:
# Ensure coverage table has 180 rows (30 companies x 6 years)
years = list(range(2019, 2025))
required_rows = [(cid, y) for cid in master['CID'] for y in years]
have_rows = set(zip(coverage['Company_Identifier'], coverage['FiscalYear'])) if not coverage.empty else set()
missing_rows = [r for r in required_rows if r not in have_rows]
len(missing_rows), missing_rows[:5]

(0, [])

In [ ]:
# Seed missing rows if needed (keeps coverage_table_updated.csv as truth)
if missing_rows:
    new_rows = []
    for cid, year in missing_rows:
        company = master.loc[master['CID'] == cid, 'CompanyName'].iloc[0]
        new_rows.append({
            'CompanyName': company,
            'Company_Identifier': cid,
            'What_to_capture': 'Annual report',
            'FiscalYear': year,
            'IR_URL': master.loc[master['CID'] == cid, 'investor_relations_url'].fillna('').iloc[0],
            'Priority': 'Not Downloaded'
        })
    coverage = pd.concat([coverage, pd.DataFrame(new_rows)], ignore_index=True)
    coverage.to_csv(coverage_path, sep='\t', index=False)
coverage.shape

(240, 27)